In [20]:
import pickle
import dotenv

In [21]:
import tqdm
import time
import re

In [22]:
from mmw.parse import *

In [23]:
dotenv.load_dotenv()

True

In [24]:
import os
import openai

openai.organization = os.getenv("OPENAI_ORGANIZATION")
openai.api_key = os.getenv("OPENAI_API_KEY")

response = openai.Completion.create(model="text-davinci-003", prompt="Say this is a test", temperature=0, max_tokens=7)
response['choices'][0]['text']

'\n\nThis is indeed a test'

In [25]:
all_parsed_essays = parse_essays()

In [26]:
marked_essays, filtered_marked_essays, all_questions = parse_annotations()

In [27]:
len(all_parsed_essays), len(filtered_marked_essays)

(77, 76)

In [28]:
#model = "text-davinci-003"
model = "ada:ft-mark-my-words-2023-05-01-11-44-26" #ada

In [29]:
import json

In [30]:
all_lines = []
with open('data/datasets/v1/finetune_set_prepared_valid.jsonl') as f:
    for line in f:
        all_lines.append(json.loads(line))

In [31]:
len(all_lines)

195

In [43]:
LIMIT_QUESTIONS = 1

In [44]:
all_answers = []
for i,line in tqdm.tqdm(enumerate(all_lines[:LIMIT_QUESTIONS])):
        
    prompt = line['prompt']

    response = openai.Completion.create(model=model, prompt=prompt, temperature=0, max_tokens=3)
    all_answers.append((0, 0, prompt, response, int(line['completion'])))
    
    time.sleep(.5)

195it [02:20,  1.39it/s]


In [45]:
len(all_answers)

195

In [41]:
all_answers[0]

(0,
 0,
 'Given is a student essay and a question to the essay. Please rate the Question with a number between 0 and 3, where 0 is "not at all satisfied" and 3 is "very satisfied.".\n\nEssay: """\nThe Cage\nIt felt like I was walking forever, i didn\'t even know were i was i just knew i was somewhere. A creepy aliway to be exazaced. To be honest i would rather\nbe at school than this nightmare well not to brag but i was not scared in the slightited, anyway desided just to keep walking to see what was down this\naliway, just as i was almost there thunder shot right though the sky it was like someone had a shotgun shot a bullet right next to my ear jezz i was not\nprepared for that, where there is thunder there is rain and i was not gonna get my new guchi shoes wet so i bulted to the end of the crusty aliway,\ni saw... well i didn\'t see anything other than cages, well i guess it was just a dream cause i don\'t own\nguchi shoes. OK enough of me being broke I gotta get to school. Time for

In [76]:
all_essays = set()
all_questions = set()
for cur in all_answers:
    cur_answer = cur[2]
    all_fount = list(re.finditer('"""', cur_answer))
    essay = cur_answer[all_fount[0].end():all_fount[1].start()]
    question = cur_answer[all_fount[2].end():all_fount[3].start()]
    
    all_essays.add(essay)
    all_questions.add(question)
    
all_essays = list(all_essays)
all_questions = list(all_questions)
for cur in range(len(all_answers)):
    cur_answer = all_answers[cur][2]
    all_fount = list(re.finditer('"""', cur_answer))
    essay = cur_answer[all_fount[0].end():all_fount[1].start()]
    question = cur_answer[all_fount[2].end():all_fount[3].start()]
    
    result = (all_essays.index(essay), all_questions.index(question), 
                        all_answers[cur][2], all_answers[cur][3], all_answers[cur][4])
    all_answers[cur] = result

In [77]:
save_results('data/evaluations/run_3_ft_ada_val', all_parsed_essays, all_questions, all_answers)

In [78]:
_,_,all_answers_loaded = load_results('data/evaluations/run_3_ft_ada_val')

In [79]:
[(a[0], a[1], a[3]['choices'][0]['text']) for a in all_answers_loaded]

[(47, 11, ' 0\n\n'),
 (47, 4, ' 1\nQuestion'),
 (62, 1, ' 3 3 3'),
 (62, 11, ' 3 3 3'),
 (62, 2, ' 3 3 3'),
 (62, 9, ' 3 3 3'),
 (14, 11, ' 3\nAnswer'),
 (14, 12, ' 3.\n'),
 (14, 9, ' 3\nAnswer'),
 (14, 10, ' 3\nAnswer'),
 (11, 11, ' 3 3 3'),
 (11, 9, ' 3 3 3'),
 (38, 1, ' 2 2 2'),
 (38, 12, ' 1. 1'),
 (38, 7, ' 2."""'),
 (38, 0, ' 2 2 2'),
 (1, 12, ' 1\nQuestion'),
 (31, 5, ' 3.3'),
 (31, 2, ' 2.3'),
 (31, 10, ' 3\nAnswer'),
 (26, 1, ' 2\nAnswer'),
 (26, 4, ' 2\nAnswer'),
 (26, 2, ' 2\nAnswer'),
 (26, 9, ' 2\nAnswer'),
 (26, 3, ' 3\nAnswer'),
 (20, 11, ' 2\n""'),
 (20, 5, ' 2\n3'),
 (48, 4, ' 2 2\n'),
 (48, 3, ' 3 3 3'),
 (32, 1, ' 2 1 2'),
 (32, 4, ' 2 1 2'),
 (32, 9, ' 2 1 2'),
 (46, 1, ' 2 2 2'),
 (46, 6, ' 2 2 2'),
 (46, 4, ' 2 2 2'),
 (46, 8, ' 1. 1'),
 (46, 7, ' 2\n2'),
 (50, 11, ' 1\nAnswer'),
 (8, 8, ' 1.1'),
 (8, 2, ' 1\nAnswer'),
 (8, 10, ' 1\nAnswer'),
 (36, 8, ' 1\nAnswer'),
 (36, 12, ' 1\nAnswer'),
 (36, 7, ' 1\nAnswer'),
 (36, 3, ' 1\nAnswer'),
 (17, 12, ' 1.1'),
 (17, 9

In [80]:
[(a[0],a[1],a[3]['choices'][0]['text']) for a in all_answers]

[(47, 11, ' 0\n\n'),
 (47, 4, ' 1\nQuestion'),
 (62, 1, ' 3 3 3'),
 (62, 11, ' 3 3 3'),
 (62, 2, ' 3 3 3'),
 (62, 9, ' 3 3 3'),
 (14, 11, ' 3\nAnswer'),
 (14, 12, ' 3.\n'),
 (14, 9, ' 3\nAnswer'),
 (14, 10, ' 3\nAnswer'),
 (11, 11, ' 3 3 3'),
 (11, 9, ' 3 3 3'),
 (38, 1, ' 2 2 2'),
 (38, 12, ' 1. 1'),
 (38, 7, ' 2."""'),
 (38, 0, ' 2 2 2'),
 (1, 12, ' 1\nQuestion'),
 (31, 5, ' 3.3'),
 (31, 2, ' 2.3'),
 (31, 10, ' 3\nAnswer'),
 (26, 1, ' 2\nAnswer'),
 (26, 4, ' 2\nAnswer'),
 (26, 2, ' 2\nAnswer'),
 (26, 9, ' 2\nAnswer'),
 (26, 3, ' 3\nAnswer'),
 (20, 11, ' 2\n""'),
 (20, 5, ' 2\n3'),
 (48, 4, ' 2 2\n'),
 (48, 3, ' 3 3 3'),
 (32, 1, ' 2 1 2'),
 (32, 4, ' 2 1 2'),
 (32, 9, ' 2 1 2'),
 (46, 1, ' 2 2 2'),
 (46, 6, ' 2 2 2'),
 (46, 4, ' 2 2 2'),
 (46, 8, ' 1. 1'),
 (46, 7, ' 2\n2'),
 (50, 11, ' 1\nAnswer'),
 (8, 8, ' 1.1'),
 (8, 2, ' 1\nAnswer'),
 (8, 10, ' 1\nAnswer'),
 (36, 8, ' 1\nAnswer'),
 (36, 12, ' 1\nAnswer'),
 (36, 7, ' 1\nAnswer'),
 (36, 3, ' 1\nAnswer'),
 (17, 12, ' 1.1'),
 (17, 9